# Financial News Generation

In this project, we will generate our own Financial News using RNNs.  We will be using Financial news published by Bloomberg.com, CNBC.com, reuters.com, wsj.com and fortune.com as our training dataset.  The Neural Network will then generate a new ,"fake" Financial News article, based on patterns it recognizes in this training dataset. Given an English sentence and the article length, the sample output of this project looks like below:

## Load the Data

The data is available in the directory as `/data/fin_news.txt`, the text file contains all Financial news published by Bloomberg, CNBC, Reuters, Wsj and Fortune in January 2018. Below, we will create a `load_data` function to load in the data and output a sample. 

In [1]:
import os
def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data


In [2]:
# load in data
data_dir = 'fin_news_2.txt'
text = load_data(data_dir)
print(text[2:1000])

17 Hours Ago | 02:56 \nEmerging markets soared more than 33 percent in 2017, and Todd Gordon of TradingAnalysis.com says the rally won\'t stop. \nA big part of the rally in emerging markets, tracked by the emerging market ETF EEM , was a weak dollar. And given that Gordon still sees the inverse relationship between EEM and the dollar, measured in his charts by the dollar-tracking ETF UUP , he believes the U.S. currency will continue to help the group. \n"We have a falling U.S. dollar, which will support international and emerging market currencies and will give those EEM stocks a boost," Gordon said Tuesday on CNBC\'s "Trading Nation." The U.S. dollar in 2017 posted its worst annual performance in 14 years, while EEM saw its best performance since 2013. \nAs for how high the latter could go, Gordon says EEM has broken "resistance" at around $45, which was the ETF\'s 2014 highs. That $45 region is now what he calls "support," and he sees it rallying to $50, which the ETF hasn\'t hit s


---
## Implement Pre-processing Functions

### Lookup Table
In dataset pre-processing, we will create a word embedding to transform the words to ids. The functions below will create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

The dictionaries will return **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    
    # counter counts unique words and subsequently sort the words
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word: i for i, word in enumerate(vocab)}
    int_to_vocab = {i: word for i, word in enumerate(vocab)}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)


### Tokenize Punctuation
We will split the text into a word array using spaces as delimiters. However, punctuations like periods and comma can create multiple ids for the same word. For example, "bullish" and "bullish!" would generate two different word ids.

The function `token_lookup` will return a dict that tokenize symbols like "?" into "Question_Mark".  The dictionary will have the symbol as key and the description as token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word.

In [4]:
def token_lookup():
    
    # tokenize dictionary where the punctuation is the key, while the description is the token
    punctuation = {'.':'<Period>', ',': '<Comma>', '"': '<Quotation_Mark>', ';': '<Semicolon>', 
                   '!': '<Exclamation_Mark>', '?': '<Question_Mark>', '(': '<Left_Parentheses>', 
                   ')': '<Right_Parentheses>', '-': '<Dash>', '\n': '<Return>'}
    
        
    return punctuation

## Pre-process all the data and save it

The code cell below will pre-process all the data and save it to file.

In [3]:
import os
import pickle
import torch


SPECIAL_WORDS = {'PADDING': '<PAD>'}


def preprocess_and_save_data(dataset_path, token_lookup, create_lookup_tables):
    
    # load text data
    text = load_data(dataset_path)
    
    # ignore brackets at the first and last 2 characters 
    text = text[2:-2]
    
    # pick only half of the data in January 2018 and preprocess the text
    text = text[0:int(len(text)/4)] 
    text = text.lower()
    text = text.replace("\\n", "")
    text = text.replace("\\", "")

    # add delimiter (space) around the token  
    token_dict = token_lookup()
    for key, token in token_dict.items():
        text = text.replace(key, ' {} '.format(token))
    
    # split text into words
    text = text.split()
    
    # create dictionaries for our vocabs and save it
    vocab_to_int, int_to_vocab = create_lookup_tables(text + list(SPECIAL_WORDS.values()))
    int_text = [vocab_to_int[word] for word in text]
    pickle.dump((int_text, vocab_to_int, int_to_vocab, token_dict), open('preprocess.p', 'wb'))





In [6]:
# pre-process training data
preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Load the vocabs
The cell below will load the previously saved preprocessed training data.

In [5]:
def load_preprocess(): 
    #Load the Preprocessed Training data
    return pickle.load(open('preprocess.p', mode='rb'))

In [6]:
int_text, vocab_to_int, int_to_vocab, token_dict = load_preprocess()

In [7]:
len(vocab_to_int)

122418

## Build the Neural Network
Build an RNN by implementing the RNN Module, forward and backpropagation functions.

### Check Access to GPU

In [8]:
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Hence CPU is used to train your neural network.')

No GPU found. Hence CPU is used to train your neural network.


## Input
For input data, use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

Define TensorDataset to accept feature and target tensors. Then create a DataLoader that accepts the data output from TensorDataset and batch size.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

Create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`, then batch words using the DataLoader. 

For example, say we have these as input:
```
int_words = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
sequence_length = 5
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4, 5]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
6
```

In [9]:
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

def batch_data(words, sequence_length, batch_size):
        
    # only full batches
    n_batches = len(words)//batch_size
    words = words[:n_batches*batch_size]
    
    x = []
    y = []
    for idx in range(0, (len(words) - sequence_length) ):
        x.append(words[idx : idx + sequence_length])
        y.append(words[idx + sequence_length])
        
    feature_tensors = torch.from_numpy(np.asarray(x))
    target_tensors = torch.from_numpy(np.asarray(y))
    data_output = TensorDataset(feature_tensors, target_tensors)
    
    # batch the neural network data using DataLoader
    dataloader = torch.utils.data.DataLoader(data_output, batch_size=batch_size)
    
    return dataloader

In [10]:
# test dataloader
test_text = range(50)
train_loader = batch_data(test_text, sequence_length=5, batch_size=10)
data_iteration = iter(train_loader)
x, y = data_iteration.next()

print(x.shape)
print(x)
print()
print(y.shape)
print(y)

torch.Size([10, 5])
tensor([[  0,   1,   2,   3,   4],
        [  1,   2,   3,   4,   5],
        [  2,   3,   4,   5,   6],
        [  3,   4,   5,   6,   7],
        [  4,   5,   6,   7,   8],
        [  5,   6,   7,   8,   9],
        [  6,   7,   8,   9,  10],
        [  7,   8,   9,  10,  11],
        [  8,   9,  10,  11,  12],
        [  9,  10,  11,  12,  13]])

torch.Size([10])
tensor([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14])


---
## Build the Neural Network
Implement an RNN using PyTorch's LSTM layers, or refer to Pytorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module) for other recurrent layers. The following functions are implemented for the class:
 - `__init__` - The initialization function
 - `init_hidden` - The initialization function for an LSTM hidden state
 - `forward` - Forward propagation function
 
The initialization function creates the layers of the neural network and then save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model is the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.


In [11]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM layers
        """
        super(RNN, self).__init__()
        
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                           dropout=dropout, batch_first=True)
        
        # define linear layer
        self.fc = nn.Linear(hidden_dim, output_size)
             
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # get batch size
        batch_size = nn_input.size(0)
        
        # embedding and LSTM output
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        # stack the outputs of the lstm to pass to the fully-connected layer 
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # fully connected layers
        output = self.fc(lstm_out)
        
        # get the last batch of word scores by shaping the output of the final and fully-connected layer
        # reshape into (batch_size, seq_length, output_size)
        out = output.view(batch_size, -1, self.output_size)
        
        # get last batch
        out = out[:, -1]

        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                    weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
            
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                     weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

### Define forward and backpropagation

The function below will run the training loop iteratively, applying forward and back propagation on the RNN class, which will then return the average loss over a batch and the hidden state returned by the RNN model.

In [12]:
import torch
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    clip = 5
    
    # move data to GPU, if available
    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()
    
    # convert the initialized hidden state to tuple to feed into the model
    h = tuple([each.data for each in hidden])
    
    # zero out the accumulated gradient 
    optimizer.zero_grad()
    
    # get output and hidden from rnn model
    output, h = rnn(inp, h)
       
    # calculate loss and perform backprop
    loss = criterion(output, target)
    loss.backward()
    nn.utils.clip_grad_norm_(rnn.parameters(), clip)
    optimizer.step()
  
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), h

## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The `train_rnn` function below will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    # switch model to train mode
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
                
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            
            # record loss
            batch_losses.append(loss)

            # printing loss statistics
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []
                save_model('./save/trained_rnn1', rnn)
                print('Model Trained and Saved')

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence
- Set `batch_size` to the batch size
- Set `num_epochs` to the number of epochs to train for
- Set `learning_rate` to the learning rate for an Adam optimizer
- Set `vocab_size` to the number of uniqe tokens in our vocabulary
- Set `output_size` to the number of output, which is the vocab_size
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size
- Set `hidden_dim` to the hidden dimension of your RNN
- Set `n_layers` to the number of layers/cells in your RNN
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress

In [22]:
# number of words in a sequence
# size of a long range dependencies the model will learn
sequence_length = 15  # of words in a sequence

# Batch Size
batch_size = 100

# get train loader for the specific batch size and sequence length
train_loader = batch_data(int_text, sequence_length, batch_size)

In [23]:
# training parameters
# number of Epochs
num_epochs = 5

# learning rate
learning_rate = 0.001

# model parameters
# vocab size
vocab_size = len(vocab_to_int)

# output size
output_size = 122418

# embedding dimension
embedding_dim = 500

# LSTM hidden dimension
hidden_dim = 1024

# number of LSTM layers
n_layers = 2

# show statistics for every n number of batches
show_every_n_batches = 5000

### Train
Next, the neural network will begin training on the pre-processed data, and then save the trained model.

In [25]:
def save_model(filename, decoder):
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save(decoder, save_filename)


In [26]:
import signal

from contextlib import contextmanager

import requests


DELAY = INTERVAL = 4 * 60  # interval time in seconds
MIN_DELAY = MIN_INTERVAL = 2 * 60
KEEPALIVE_URL = "https://nebula.udacity.com/api/v1/remote/keep-alive"
TOKEN_URL = "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token"
TOKEN_HEADERS = {"Metadata-Flavor":"Google"}


def _request_handler(headers):
    def _handler(signum, frame):
        requests.request("POST", KEEPALIVE_URL, headers=headers)
    return _handler


@contextmanager
def active_session(delay=DELAY, interval=INTERVAL):
    """
    Example:

    from workspace_utils import active session

    with active_session():
        # do long-running work here
    """
    token = requests.request("GET", TOKEN_URL, headers=TOKEN_HEADERS).text
    headers = {'Authorization': "STAR " + token}
    delay = max(delay, MIN_DELAY)
    interval = max(interval, MIN_INTERVAL)
    original_handler = signal.getsignal(signal.SIGALRM)
    try:
        signal.signal(signal.SIGALRM, _request_handler(headers))
        signal.setitimer(signal.ITIMER_REAL, delay, interval)
        yield
    finally:
        signal.signal(signal.SIGALRM, original_handler)
        signal.setitimer(signal.ITIMER_REAL, 0)


def keep_awake(iterable, delay=DELAY, interval=INTERVAL):
    """
    Example:

    from workspace_utils import keep_awake

    for i in keep_awake(range(5)):
        # do iteration with lots of work here
    """
    with active_session(delay, interval): yield from iterable

In [ ]:


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
#torch.backends.cudnn.enabled = False
with active_session():
    
    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
    #try:
    #    rnn.cuda()
    #except:
    #    rnn.cuda()
    if train_on_gpu:
        rnn.cuda()

    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    save_model('./save/trained_RNN', trained_rnn)
    print('Model Trained and Saved')

Training for 6 epoch(s)...
Epoch:    1/6     Loss: 7.310709282875061

Epoch:    1/6     Loss: 6.656816635322571

Epoch:    1/6     Loss: 6.352073802375793

Epoch:    1/6     Loss: 6.296273772144318

Epoch:    1/6     Loss: 6.373035291481018

Epoch:    1/6     Loss: 6.1749204139709475

Epoch:    1/6     Loss: 6.1028848300457

Epoch:    1/6     Loss: 6.050500279617309

Epoch:    1/6     Loss: 5.973662206602096

Epoch:    1/6     Loss: 5.901190425682068

Epoch:    1/6     Loss: 5.935687744808197



In [ ]:
#torch.backends.cudnn.enabled = False
with active_session():
    
    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
    #try:
    #    rnn.cuda()
    #except:
    #    rnn.cuda()
    if train_on_gpu:
        rnn.cuda()

    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    save_model('./save/trained_RNN', trained_rnn)
    print('Model Trained and Saved')

Training for 5 epoch(s)...
Epoch:    1/5     Loss: 9.393236272621154



/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved
Epoch:    1/5     Loss: 8.774122006320953

Model Trained and Saved
Epoch:    1/5     Loss: 9.197710079097748

Model Trained and Saved
Epoch:    1/5     Loss: 9.247907335948945

Model Trained and Saved


In [ ]:
#torch.backends.cudnn.enabled = False
with active_session():
    
    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
    #try:
    #    rnn.cuda()
    #except:
    #    rnn.cuda()
    if train_on_gpu:
        rnn.cuda()

    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    save_model('./save/trained_RNN', trained_rnn)
    print('Model Trained and Saved')

Training for 5 epoch(s)...


### How I decide on the model hyperparameters? 


**Answer:** To reach for our goal of less than 3.5 in loss, we need to run a training of at least 10 epochs. Increasing the hidden dimension from 512 to 1024 has proven to converge faster with loss of 3 versus 2.5 after 10 epochs. The ideal n_layers was found to be 2. Increasing the n_layer to 3 or higher did not make the model converge faster, and it actually did worse with loss value oscillating between 6 and 7 even after 5 epochs. The ideal learn rate was found to be 0.001, if 0.01 learn rate was used, the gradient descent overshoots causing the loss to increase by each epochs. If a much lower 0.0001 was used in training, the converge would be too slow, hence very inefficient. Changing the embedding dimension from 500 to 800 did not make any significant improvement in the model, so we will stick to the lower 500 embedding dimension. Increasing the sequence length to 50 made the model to converge slower, and it took almost 4 hours to finish running just 1 epoch. In our case we will use the average English sentences length, which is 15 words per sentence, hence sequence length is set to 15.   

---
# Load trained model

After saving our trained model by name, `trained_RNN`, below cell will load in our word:id dictionaries and our trained RNN model.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
else:
    map_location='cpu'

_, vocab_to_int, int_to_vocab, token_dict = load_preprocess()
trained_rnn = torch.load('trained_rnn.pt', map_location=map_location)

## Generate Financial News
With the network trained and saved, we can then generate a new, "fake" Financial News in this section.

### Generate Text
To generate the text, the network will start predictions with the last single word of a user input sentence, and repeat its predictions until it reaches a set length. The `generate` function below will take a word id, `prime_id` to start with, and generates a set length of text, `predict_len`. Also, we use topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [30]:
import torch.nn.functional as F
import numpy as np

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation descriptions
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    # swith model to evaluation mode
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
          
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
       
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
       
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()  
        word_i = np.random.choice(top_i, p=p/p.sum())
 
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Financial News
It's time to generate the text. Set `gen_length` to the length of Financial news article you want to generate and input sentence into `prime_sentence` to start the prediction.

If the last word in your sentence cannot be recognized by vocab_to_int dictionary, the prediction will fail, please try again with other word.  

In [32]:
# run the cell multiple times to get different results!
gen_length = 500 # modify the length to your preference
prime_word = 'george' # name for starting the script

#SPECIAL_WORDS = {'PADDING': '<PAD>'}
pad_word = SPECIAL_WORDS['PADDING']
generated_article = generate(trained_rnn, vocab_to_int[prime_word +":"], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_article)

george: it's a whore!

george: i think it's neat.

jerry: oh, i don't want to interrupt anymore.

elaine: you know, i think it's a beautiful contest. it's a beautiful thing!

jerry: hey, hey. you know what? you are so funny.

george: oh, no.

jerry: what?

kramer: oh, come on!

jerry: what? what is that noise?

elaine: no, no, no. it's just a cake present.

hoyt: so, what's wrong with you?

elaine: yeah, yeah.

jerry: what?

cindy: no, it was a mistake.

george: oh my god!

elaine: i don't know if i'm going to hell! i am so glad that i was a philanthropist girl, a member of my own opium excursions.

jerry: what is that noise?

jerry: yeah. it's just as much as we used to make a big difference. i'm a fancy person! it's a beautiful name.

hoyt: so?

elaine: i think that's the end.

elaine: what? what is that noise?

jerry: yeah, and then you have a lot of money.

elaine: you can't eat this thing.

george: what is your name again?

jerry: i don't know.

george: you know, the people who ha

#### Save your Financial News article

Once you have an article that you like (or find interesting), the cell below will save the article to a text file!

In [24]:
# save script to a text file
f =  open("generated_article_1.txt","w")
f.write(generated_article)
f.close()

# The Financial News is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 
